In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.amazon.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.amazon.com/')

True

In [10]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [11]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe')
driver.maximize_window()

In [13]:
driver.get('https://tiki.vn')
time.sleep(1.)
driver.get('https://amazon.com')

In [14]:
elem = driver.find_element_by_id('twotabsearchtextbox')
elem.clear()
elem.send_keys("laptop")
elem.send_keys(Keys.RETURN)